In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [64]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [105]:
data_df= pd.read_csv("Combined_News_DJIA.csv")

In [106]:
data_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [108]:
#df['period'] = df[['Year', 'quarter']].apply(lambda x: ''.join(x), axis=1)
'''
top_news_list= ['Top1','Top2','Top3','Top4','Top5','Top6','Top7','Top8','Top9','Top10',
             'Top11','Top12','Top13','Top14','Top15','Top16','Top17','Top18','Top19','Top20',
             'Top21','Top22','Top23','Top24','Top25']
'''

top_news_list= ['Top1','Top2','Top3','Top4','Top5','Top6','Top7','Top8','Top9','Top10',
               'Top11','Top12','Top13','Top14','Top15','Top16','Top17','Top18','Top19','Top20',
               'Top21','Top22']

data_df['TopNews'] = data_df[top_news_list].apply(lambda x: ','.join(x), axis=1)

# for some reasons above command is not concatenating Top23-Top25
data_df['TopNews']= data_df['TopNews']+','+ data_df['Top23']+','+data_df['Top24']+ ',' + data_df['Top25']

In [109]:
data_df['TopNews'][2]

'b\'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.\',b"Russia \'ends Georgia operation\'",b\'"If we had no sexual harassment we would have no children..."\',b"Al-Qa\'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers",b\'Ceasefire in Georgia: Putin Outmaneuvers the West\',b\'Why Microsoft and Intel tried to kill the XO $100 laptop\',b\'Stratfor: The Russo-Georgian War and the Balance of Power   \',b"I\'m Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did",b"The US military was surprised by the timing and swiftness of the Russian military\'s move into South Ossetia and is still trying to sort out what happened, a US defense official said Monday",b\'U.S. Beats War Drum as Iran Dumps the Dollar\',b\'Gorbachev: "Georgian military attacked the South Ossetian capital of Tskhinvali with multi

In [110]:
merged_df= data_df[['Date','Label','TopNews']]

In [111]:
merged_df.head()

,Date,Label,TopNews
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...
4,2008-08-14,1,b'All the experts admit that we should legalis...


In [112]:
def news_to_words (raw_news):
    
    # Remove the HTML characters
    news_text = BeautifulSoup(raw_news)
    
    # Remove the punctuals
    letters_only = re.sub("[^a-zA-Z0-9.]"," ",news_text.get_text())
    
    # Convert it in lower case and split them into words
    lower_case = letters_only.lower()
    words = lower_case.split()
    
    #In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english")) 
    
    meaningfull_words = [w for w in words if not w in stops]
    
    
    #Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningfull_words ))

In [113]:
clean_news= news_to_words(merged_df['TopNews'][2])
clean_news

/Users/rajegupt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/rajegupt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


'b remember adorable 9 year old sang opening ceremonies fake too. b russia ends georgia operation b sexual harassment would children... b al qa eda losing support iraq brutal crackdown activities regards un islamic including women buying cucumbers b ceasefire georgia putin outmaneuvers west b microsoft intel tried kill xo 100 laptop b stratfor russo georgian war balance power b trying get sense whole georgia russia war vote think georgia started think russia b us military surprised timing swiftness russian military move south ossetia still trying sort happened us defense official said monday b u.s. beats war drum iran dumps dollar b gorbachev georgian military attacked south ossetian capital tskhinvali multiple rocket launchers designed devastate large areas b cnn use footage tskhinvali ruins cover georgian report video b beginning war olympics opening violates ancient tradition truce conflict games. ioc could respond taking 2014 games away russia. b 55 pyramids large luxor stacked meg

In [114]:
num_newsdays= merged_df['TopNews'].size
num_newsdays

1989

In [115]:
clean_news_all= []

for i in range(0, num_newsdays):
    # If the index is evenly divisible by 400, print a message
    if( (i+1)%400 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_newsdays))
    
    clean_news_all.append(news_to_words(str(merged_df['TopNews'][i])))

/Users/rajegupt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/rajegupt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 400 of 1989

Review 800 of 1989

Review 1200 of 1989

Review 1600 of 1989



In [117]:
clean_news_all[2]

'b remember adorable 9 year old sang opening ceremonies fake too. b russia ends georgia operation b sexual harassment would children... b al qa eda losing support iraq brutal crackdown activities regards un islamic including women buying cucumbers b ceasefire georgia putin outmaneuvers west b microsoft intel tried kill xo 100 laptop b stratfor russo georgian war balance power b trying get sense whole georgia russia war vote think georgia started think russia b us military surprised timing swiftness russian military move south ossetia still trying sort happened us defense official said monday b u.s. beats war drum iran dumps dollar b gorbachev georgian military attacked south ossetian capital tskhinvali multiple rocket launchers designed devastate large areas b cnn use footage tskhinvali ruins cover georgian report video b beginning war olympics opening violates ancient tradition truce conflict games. ioc could respond taking 2014 games away russia. b 55 pyramids large luxor stacked meg

In [142]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

clean_news_all_1= [[porter_stemmer.stem(word) for word in sentence.split(" ")] for sentence in clean_news_all]
clean_news_all_1= [" ".join(sentence) for sentence in clean_news_all_1]

In [143]:
print ("creating the bag of words")

from sklearn.feature_extraction.text import CountVectorizer

# Played with different max_features hyperparameter
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None)
                             #max_features=100)


# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.

train_data_features = vectorizer.fit_transform(clean_news_all_1)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

print ((train_data_features).shape)

print (train_data_features)

creating the bag of words
(1989, 26637)
[[0 1 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 1 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [131]:
train_data_features[1][:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 3, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 7, 2, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 3, 0, 0, 1, 0, 0], dtype=int64)

In [132]:
vocab= vectorizer.get_feature_names()
print (vocab)

['000', 'anti', 'arrest', 'attack', 'back', 'ban', 'bank', 'bomb', 'british', 'call', 'canada', 'children', 'china', 'chines', 'citi', 'claim', 'could', 'countri', 'court', 'day', 'dead', 'death', 'die', 'drug', 'face', 'fire', 'first', 'forc', 'found', 'gaza', 'german', 'germani', 'get', 'govern', 'group', 'human', 'india', 'intern', 'iran', 'iraq', 'islam', 'isra', 'israel', 'japan', 'kill', 'korea', 'last', 'law', 'leader', 'like', 'make', 'man', 'may', 'militari', 'million', 'minist', 'muslim', 'nation', 'new', 'news', 'north', 'nuclear', 'offici', 'oil', 'old', 'one', 'pakistan', 'palestinian', 'peopl', 'plan', 'polic', 'power', 'presid', 'protest', 'report', 'right', 'russia', 'russian', 'said', 'saudi', 'say', 'show', 'south', 'state', 'strike', 'syria', 'take', 'time', 'two', 'uk', 'un', 'us', 'use', 'want', 'war', 'warn', 'women', 'world', 'would', 'year']


In [135]:
print (len(merged_df[merged_df['Label']==0]))
print (len(merged_df[merged_df['Label']==1]))

924
1065


In [136]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data_features, merged_df['Label'], test_size =0.2, random_state=0)

In [137]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = RandomForestClassifier(max_depth=2, random_state=0)
RF_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [138]:
RFpred_train= RF_clf.predict(X_train)
RFpred_test= RF_clf.predict(X_test)

In [139]:
from sklearn.metrics import roc_auc_score

print ("train score: ", roc_auc_score(y_train,RFpred_train))
print ("test score: ", roc_auc_score(y_test,RFpred_test))

train score:  0.543775649795
test score:  0.505168709718


In [140]:
from sklearn.linear_model import LogisticRegression
LR_clf= LogisticRegression()
LR_clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [141]:
LRpred_train= LR_clf.predict(X_train)
LRpred_test= LR_clf.predict(X_test)

print ("train score: ", roc_auc_score(y_train,LRpred_train))
print ("test score: ", roc_auc_score(y_test,LRpred_test))

train score:  0.597058823529
test score:  0.508024769367
